### 뉴발란스 랭킹 신발들의 정보를 수집해보자

In [1]:
from selenium import webdriver as wb
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from tqdm import tqdm
import pandas as pd
import time

In [3]:
driver = wb.Chrome()
driver.maximize_window()
time.sleep(2)
driver.get('https://www.nbkorea.com/product/featuredList.action?cateGrpCode=250100&cIdx=1385')

### 물품의 이미지를 클릭해서 정보가 있는 페이지로 이동

In [161]:
img = driver.find_element(By.CSS_SELECTOR,'img#selGoods')

In [163]:
img.click()

### 상품 세부사항 크롤링
- 상품명
- 평점
- 구매가능한 사이즈, 구매불가능한 사이즈
- 발볼넓이
- 리뷰개수(숫자만)

In [166]:
name = driver.find_element(By.CSS_SELECTOR,'#displayName')
rating = driver.find_element(By.CSS_SELECTOR,'.crema_product_reviews_score__container')
rating_num = driver.find_element(By.CSS_SELECTOR,'.review.crema-product-reviews-count.crema-applied')
size = driver.find_elements(By.CSS_SELECTOR,'#optSizeSection .items > li > input')
price = driver.find_element(By.CSS_SELECTOR,'.won')
foot_width = driver.find_element(By.CSS_SELECTOR,'#optWidthSection > ul > li > label')

In [168]:
# 사이즈가 input 태그 안에 있는데, 사이즈가 없는 것에는 disabled 속성이 있다.
size_avail = driver.find_element(By.CSS_SELECTOR,'#optSizeSection > ul > li > input')
# get_attribute: 특정 요소 내 속성을 검색하는 함수
# 사이즈가 있는 것들은 아무것도 출력이 안 된다.
size_avail.get_attribute('disabled')

size_unavail = driver.find_element(By.CSS_SELECTOR,'#size_290')
# 290 사이즈는 없기 때문에 true가 출력된다!
size_unavail.get_attribute('disabled')

'true'

### CSS_SELECTOR로 태그의 속성에 접근하는 방법
- 태그명[속성 = '~']

In [171]:
# <label for="size_290" title="290">290</label>
temp = driver.find_element(By.CSS_SELECTOR,'label[for = "size_290"]')
temp.text

'290'

In [173]:
size_avail_list, size_unavail_list= [], []

for i in size:
    input_id = i.get_attribute('id')
    # lable태그의 for 속성에 들어갈 size_~는 id값에 동일하게 있으므로 아래에서 활용하기 위해 추출
    is_disabled = i.get_attribute('disabled')
    # label의 for 속성에 접근
    label = driver.find_element(By.CSS_SELECTOR, f'label[for = "{input_id}"]')
    label_text = label.text.strip()
    
    if is_disabled:
        size_unavail_list.append(label_text)
    else:
        size_avail_list.append(label_text)

print(size_avail_list)
print(size_unavail_list)

['220', '225', '230', '235', '240', '245', '250', '255', '260', '270', '275', '280']
['265', '285', '290']


In [185]:
print(name.text)
print(rating.text)
print(rating_num.text[1:-1] + '개')
print(price.text)
print(foot_width.text)

M1906REH
4.9
1,312개
179,000원


D(보통)


### 반복문을 활용해서 상위 4개 신발의 세부사항 수집해서 출력하기

In [7]:
for i in tqdm(range(4)):
    img = driver.find_elements(By.CSS_SELECTOR,'img#selGoods') # 여기다 써야 반복문이 돌아갈 때마다 이미지에 접근한다!
    img[i].click()
    time.sleep(2)
    name = driver.find_element(By.CSS_SELECTOR,'#displayName')
    rating = driver.find_element(By.CSS_SELECTOR,'.crema_product_reviews_score__container')
    rating_num = driver.find_element(By.CSS_SELECTOR,'.review.crema-product-reviews-count.crema-applied')
    size = driver.find_elements(By.CSS_SELECTOR,'#optSizeSection .items > li > input')
    price = driver.find_element(By.CSS_SELECTOR,'.won')
    foot_width = driver.find_element(By.CSS_SELECTOR,'#optWidthSection > ul > li > label')

    size_avail_list, size_unavail_list= [], []

    for i in size:
        input_id = i.get_attribute('id')
        # lable태그의 for 속성에 들어갈 size_~는 id값에 동일하게 있으므로 아래에서 활용하기 위해 추출
        is_disabled = i.get_attribute('disabled')
        # label의 for 속성에 접근
        label = driver.find_element(By.CSS_SELECTOR, f'label[for = "{input_id}"]')
        label_text = label.text.strip()
        
        if is_disabled:
            size_unavail_list.append(label_text)
        else:
            size_avail_list.append(label_text)
    
        
    print(name.text)
    print(rating.text)
    print(rating_num.text[1:-1] + '개')
    print(price.text)
    print(foot_width.text)
    print(size_avail_list)
    print(size_unavail_list)
    
    driver.back()
    time.sleep(2)

  0%|                                                                                            | 0/4 [00:00<?, ?it/s]

M1906REH
4.9
1,312개
179,000원
D(보통)
['220', '225', '230', '235', '240', '245', '250', '255', '260', '270', '275', '280']
['265', '285', '290']


 25%|█████████████████████                                                               | 1/4 [00:07<00:23,  7.80s/it]

U740NW2 (WHITE SILVER METALLIC)
5.0
4개
139,000원
D(보통)
['220', '225', '230', '235', '240', '245', '250', '255', '260', '265', '270', '275', '280', '285', '300']
['290']


 50%|██████████████████████████████████████████                                          | 2/4 [00:16<00:16,  8.21s/it]

Kave / SD2401BE
4.9
130개
149,000원
M(보통)
['230', '240', '250', '260', '270', '280']
[]


 75%|███████████████████████████████████████████████████████████████                     | 3/4 [00:23<00:07,  7.55s/it]

MR530GMT
4.9
502개
129,000원
D(보통)
['220', '225', '230', '245', '255', '275']
['235', '240', '250', '260', '265', '270', '280', '285', '290', '300']


100%|████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:30<00:00,  7.67s/it]


In [272]:
driver.quit()

### 한 셀에 모든 코드 넣기
- 상위 8개의 신발 품목 데이터를 수집해서 데이터프레임화 시켜보자.

In [55]:
driver = wb.Chrome()
driver.maximize_window()
driver.get('https://www.nbkorea.com/product/featuredList.action?cateGrpCode=250100&cIdx=1385')
time.sleep(2)

# 데이터프레임으로 만들 리스트 생성
name_list, rating_list, rating_num_list, price_list, size_avail_list, foot_width_list = [],[],[],[],[],[]

try:
    for i in tqdm(range(8)):
        img = driver.find_elements(By.CSS_SELECTOR,'img#selGoods') # 여기다 써야 반복문이 돌아갈 때마다 이미지에 접근한다!
        img[i].click()
        time.sleep(2)
        
        name = driver.find_element(By.CSS_SELECTOR,'#displayName')
        rating = driver.find_element(By.CSS_SELECTOR,'.crema_product_reviews_score__container')
        rating_num = driver.find_element(By.CSS_SELECTOR,'.review.crema-product-reviews-count.crema-applied')
        size = driver.find_elements(By.CSS_SELECTOR,'#optSizeSection .items > li > input')
        price = driver.find_element(By.CSS_SELECTOR,'.won')
        foot_width = driver.find_element(By.CSS_SELECTOR,'#optWidthSection > ul > li > label')

        name_list.append(name.text)
        rating_list.append(rating.text)
        rating_num_list.append(rating_num.text[1:-1] + "개")
        price_list.append(price.text)
        foot_width_list.append(foot_width.text)
        
        size_avail = []
    
        for i in size:
            input_id = i.get_attribute('id')
            # lable태그의 for 속성에 들어갈 size_~는 id값에 동일하게 있으므로 아래에서 활용하기 위해 추출
            is_disabled = i.get_attribute('disabled')
            # label의 for 속성에 접근
            label = driver.find_element(By.CSS_SELECTOR, f'label[for = "{input_id}"]')
            label_text = label.text.strip()
            
            if not is_disabled:
                size_avail.append(label_text)

        size_avail_list.append(size_avail)
        driver.back()
        time.sleep(2)
except:
    print('error occurred')

print('number of data collected: ', len(name_list),  len(price_list), len(rating_list), len(rating_num_list), len(size_avail_list), len(foot_width_list))

driver.quit()

shoes_dic = {"shoe name": name_list,
             "price": price_list,
             "ratings":rating_list,
             "rating count":rating_num_list,
             "available size":size_avail_list,
             "foot width":foot_width_list}
shoes_df = pd.DataFrame(shoes_dic)
shoes_df.index = range(1,len(name_list)+1)
shoes_df.index.name = 'No'
shoes_df

100%|████████████████████████████████████████████████████████████████████████████████████| 8/8 [00:58<00:00,  7.31s/it]


number of data collected:  8 8 8 8 8 8


,shoe name,price,ratings,rating count,available size,foot width
No,,,,,,
1,M1906REH,"179,000원",4.9,"1,312개","[220, 225, 230, 235, 240, 245, 250, 255, 260, 270, 275, 280]",D(보통)
2,U740NW2 (WHITE SILVER METALLIC),"139,000원",5.0,4개,"[220, 225, 230, 235, 240, 245, 250, 255, 260, 265, 270, 275, 280, 285, 300]",D(보통)
3,Kave / SD2401BE,"149,000원",4.9,130개,"[230, 240, 250, 260, 270, 280]",M(보통)
4,MR530GMT,"129,000원",4.9,502개,"[220, 225, 230, 235, 245, 255, 275]",D(보통)
5,"프레쉬폼x 1080 V14 (남성, 2E)","199,000원",4.9,587개,"[250, 255, 260, 265, 270, 275, 280, 285, 290, 295]",2E(약간 넓음)
6,MR530DAM,"129,000원",4.9,297개,"[220, 225, 230, 235, 240]",D(보통)
7,"프레쉬폼x 1080 V14 (남성, 2E)","199,000원",4.9,587개,"[250, 255, 260, 265, 270, 275, 280, 285, 290]",2E(약간 넓음)
8,U740NC2 (NEW SPRUCE CLAY ASH),"139,000원",5.0,4개,"[220, 225, 230, 235, 240, 245, 250, 255, 260, 265, 270, 275, 280, 285, 290]",D(보통)


In [53]:
driver = wb.Chrome()
driver.maximize_window()
time.sleep(2)
driver.get('https://www.nbkorea.com/product/featuredList.action?cateGrpCode=250100&cIdx=1385')
time.sleep(2)

# 데이터프레임으로 만들 리스트 생성
name_list, rating_list, rating_num_list, price_list, foot_width_list = [],[],[],[],[]

try:
    for i in tqdm(range(8)):
        img = driver.find_elements(By.CSS_SELECTOR,'img#selGoods') # 여기다 써야 반복문이 돌아갈 때마다 이미지에 접근한다!
        img[i].click()
        time.sleep(2)
        
        name = driver.find_element(By.CSS_SELECTOR,'#displayName')
        name_list.append(name.text)
        
        rating = driver.find_element(By.CSS_SELECTOR,'.crema_product_reviews_score__container')
        rating_list.append(rating.text)
        
        rating_num = driver.find_element(By.CSS_SELECTOR,'.review.crema-product-reviews-count.crema-applied')
        rating_num_list.append(rating_num.text[1:-1] + "개")
        
        price = driver.find_element(By.CSS_SELECTOR,'.won')
        price_list.append(price.text)
        
        foot_width = driver.find_element(By.CSS_SELECTOR,'#optWidthSection > ul > li > label')
        foot_width_list.append(foot_width.text)
        
        driver.back()
        time.sleep(2)
except:
    print('error occurred')

print('number of data collected: ', len(name_list), len(rating_list), len(rating_num_list),  len(price_list), len(foot_width_list))

driver.quit()

shoes_dic = {"shoe name": name_list,
             "price": price_list,
             "ratings":rating_list,
             "rating count":rating_num_list,
             "foot width":foot_width_list}
shoes_df = pd.DataFrame(shoes_dic)
shoes_df.index = range(1,len(name_list)+1)
shoes_df.index.name = 'No'
shoes_df

100%|████████████████████████████████████████████████████████████████████████████████████| 8/8 [00:59<00:00,  7.45s/it]


number of data collected:  8 8 8 8 8


,shoe name,price,ratings,rating count,foot width
No,,,,,
1,M1906REH,"179,000원",4.9,"1,312개",D(보통)
2,U740NW2 (WHITE SILVER METALLIC),"139,000원",5.0,4개,D(보통)
3,Kave / SD2401BE,"149,000원",4.9,130개,M(보통)
4,MR530GMT,"129,000원",4.9,502개,D(보통)
5,"프레쉬폼x 1080 V14 (남성, 2E)","199,000원",4.9,587개,2E(약간 넓음)
6,MR530DAM,"129,000원",4.9,297개,D(보통)
7,"프레쉬폼x 1080 V14 (남성, 2E)","199,000원",4.9,587개,2E(약간 넓음)
8,U740NC2 (NEW SPRUCE CLAY ASH),"139,000원",5.0,4개,D(보통)


In [49]:
# 강사님 코드

driver = wb.Chrome()
driver.maximize_window()
time.sleep(2)
driver.get("https://www.nbkorea.com/product/featuredList.action?cateGrpCode=250100&cIdx=1385")
time.sleep(2)
# 각 상품들의 세부정보를 담아줄 빈 리스트 생성
title_list, mean_rating_review_list, enabled_size_list, foot_width_list, cnt_reviews_list = [], [], [], [], []
try :
    for i in tqdm(range(8)) :
        imgs = driver.find_elements(By.CSS_SELECTOR, "a.pro_area > img.img_goods")
        imgs[i].click()
        time.sleep(2)
        title = driver.find_element(By.CSS_SELECTOR, 'h2.title')
        title_list.append(title.text)
        mean_rating_review = driver.find_element(By.CSS_SELECTOR, 'div.crema_product_reviews_score__container')
        mean_rating_review_list.append(mean_rating_review.text)
        size = driver.find_elements(By.CSS_SELECTOR, '#optSizeSection > ul.items > li > input')
        enabled_size, disabled_size = [], []
        for j in size :
            input_id = j.get_attribute("id")
            is_disabled = j.get_attribute("disabled")
            label = driver.find_element(By.CSS_SELECTOR, f'label[for="{input_id}"]')
            label_text = label.text.strip()
            if is_disabled :
                disabled_size.append(label_text)
            else :
                enabled_size.append(label_text)
        enabled_size_list.append(enabled_size)   # 구매가능한 사이즈는 원래 리스트였으므로 text없이 그냥 리스트로 리스트에 추가
        foot_width = driver.find_element(By.CSS_SELECTOR, '#optWidthSection > ul > li > label')
        foot_width_list.append(foot_width.text)
        cnt_reviews = driver.find_element(By.CSS_SELECTOR, 'a.crema-product-reviews-count.crema-applied')
        cnt_reviews_list.append(cnt_reviews.text[1:-1]+'개')
        driver.back()
        time.sleep(2)
except :
    print("데이터 수집 중 예기치 못한 문제 발생!")
print("수집된 데이터 개수 확인 :")
print(len(title_list), len(mean_rating_review_list), len(cnt_reviews_list), len(enabled_size_list), len(foot_width_list))
driver.quit()
# 데이터 프레임 작업
newbal_dict = {"상품명":title_list, "리뷰 평점":mean_rating_review_list, "리뷰 개수":cnt_reviews_list,
               "구매가능 사이즈":enabled_size_list, "발볼":foot_width_list}
newbal_df = pd.DataFrame(newbal_dict, index=range(1,len(title_list)+1))
newbal_df.index.name = "No"
pd.set_option('display.max_colwidth', None)
newbal_df

100%|████████████████████████████████████████████████████████████████████████████████████| 8/8 [00:58<00:00,  7.34s/it]


수집된 데이터 개수 확인 :
8 8 8 8 8


,상품명,리뷰 평점,리뷰 개수,구매가능 사이즈,발볼
No,,,,,
1,M1906REH,4.9,"1,312개","[220, 225, 230, 235, 240, 245, 250, 255, 260, 270, 275, 280]",D(보통)
2,U740NW2 (WHITE SILVER METALLIC),5.0,4개,"[220, 225, 230, 235, 240, 245, 250, 255, 260, 265, 270, 275, 280, 285, 300]",D(보통)
3,Kave / SD2401BE,4.9,130개,"[230, 240, 250, 260, 270, 280]",M(보통)
4,MR530GMT,4.9,502개,"[220, 225, 230, 235, 245, 255, 275]",D(보통)
5,"프레쉬폼x 1080 V14 (남성, 2E)",4.9,586개,"[250, 255, 260, 265, 270, 275, 280, 285, 290, 295]",2E(약간 넓음)
6,MR530DAM,4.9,297개,"[220, 225, 230, 235, 240]",D(보통)
7,"프레쉬폼x 1080 V14 (남성, 2E)",4.9,586개,"[250, 255, 260, 265, 270, 275, 280, 285, 290]",2E(약간 넓음)
8,U740NC2 (NEW SPRUCE CLAY ASH),5.0,4개,"[220, 225, 230, 235, 240, 245, 250, 255, 260, 265, 270, 275, 280, 285, 290]",D(보통)
